In [18]:
import numpy as np
import pickle

# 加载数据
with open('R_ET_1_ET_2_ET_3_0.pkl', 'rb') as file:
    data = pickle.load(file)
print(data)

# 提取9x9的 Fisher 矩阵
fisher_matrix_1 = data[1]
fisher_matrix_2 = data[2][0]
fisher_matrix_3 = data[2][1]

[array([135.47015588, 144.68392831, 226.74049058]), array([[ 4.88439419e+06,  6.28884176e+05,  3.94922216e+08,
         1.58546254e+04,  1.30809358e+06, -5.33741930e+09,
        -7.25350220e+03, -5.28083064e+09, -5.14043623e+05],
       [ 6.28884176e+05,  1.02318971e+06, -1.50416305e+08,
        -4.58361328e+04,  2.75853785e+05, -1.52930408e+09,
         2.08454070e+04, -1.55837793e+09,  9.59233631e+03],
       [ 3.94922216e+08, -1.50416305e+08,  7.52393549e+10,
        -3.27933212e+04,  8.27921534e+07, -2.60342498e+11,
         2.35227745e+04, -2.48092420e+11, -5.81903462e+07],
       [ 1.58546254e+04, -4.58361328e+04, -3.27933212e+04,
         4.80865364e+05,  2.96859771e+01, -4.78015116e+05,
        -2.16041829e+05, -5.20066374e+05,  6.72039678e+00],
       [ 1.30809358e+06,  2.75853785e+05,  8.27921534e+07,
         2.96859771e+01,  3.62576635e+05, -1.52859861e+09,
        -6.30912622e+00, -1.51766899e+09, -1.29054804e+05],
       [-5.33741930e+09, -1.52930408e+09, -2.60342498e+11,

In [19]:
# 动态正则化项函数
def get_dynamic_epsilon(matrix):
    cond_number = np.linalg.cond(matrix)
    eigvals = np.linalg.eigvals(matrix)
    eigval_std = np.std(eigvals)
    
    if cond_number > 1e20 or eigval_std > 1e2:
        return 1e-2
    elif cond_number > 1e16 or eigval_std > 1e1:
        return 1e-3
    elif cond_number > 1e12:
        return 1e-4
    else:
        return 1e-6

# 强化特征值裁剪，确保特征值在更严格的范围内
def ensure_positive_definite(matrix, matrix_name=""):
    eigvals, eigvecs = np.linalg.eigh(matrix)
    
    # 打印特征值供调试
    print(f"{matrix_name} Eigenvalues before clipping: {eigvals}")
    
    # 将特征值限制在 [1e-2, 1e3] 之间，避免极端特征值
    eigvals_clipped = np.clip(eigvals, 1e-2, 1e3)
    
    # 打印裁剪后的特征值
    print(f"{matrix_name} Eigenvalues after clipping: {eigvals_clipped}")
    
    # 重构正定矩阵
    matrix_pos_def = eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T
    return matrix_pos_def

# 正则化矩阵
def regularize_matrix(matrix, matrix_name=""):
    epsilon = get_dynamic_epsilon(matrix)
    
    # 打印条件数供调试
    cond_number = np.linalg.cond(matrix)
    if cond_number > 1e12:
        print(f"Matrix condition number is high ({cond_number}), adding regularization with epsilon {epsilon}.")
    
    # 确保矩阵对称性
    matrix_sym = (matrix + matrix.T) / 2
    return ensure_positive_definite(matrix_sym + epsilon * np.eye(matrix.shape[0]), matrix_name)

In [20]:
# 计算协方差矩阵
def compute_covariance(fisher_matrix, matrix_name=""):
    reg_matrix = regularize_matrix(fisher_matrix, matrix_name)
    cov_matrix = np.linalg.pinv(reg_matrix)
    return cov_matrix

cov_matrix_1 = compute_covariance(fisher_matrix_1, "matrix_1")
cov_matrix_2 = compute_covariance(fisher_matrix_2, "matrix_2")
cov_matrix_3 = compute_covariance(fisher_matrix_3, "matrix_3")

# 打印验证
print("Shape of fisher_matrix_1:", fisher_matrix_1.shape)
print("Shape of cov_matrix_1:", cov_matrix_1.shape)

Matrix condition number is high (49331252765552.74), adding regularization with epsilon 0.01.
matrix_1 Eigenvalues before clipping: [1.13565569e+00 3.66735504e+02 6.96719573e+02 1.52860959e+05
 6.27368455e+05 2.51884792e+06 2.47787916e+09 7.45020720e+10
 5.54883037e+13]
matrix_1 Eigenvalues after clipping: [   1.13565569  366.73550391  696.71957299 1000.         1000.
 1000.         1000.         1000.         1000.        ]
Matrix condition number is high (2.6226194699865115e+21), adding regularization with epsilon 0.01.
matrix_2 Eigenvalues before clipping: [9.34878557e-03 1.02539811e-02 1.15131793e-02 2.85227322e+04
 1.12816508e+05 1.18491127e+05 4.62886377e+05 1.31671512e+08
 4.35817983e+13]
matrix_2 Eigenvalues after clipping: [1.00000000e-02 1.02539811e-02 1.15131793e-02 1.00000000e+03
 1.00000000e+03 1.00000000e+03 1.00000000e+03 1.00000000e+03
 1.00000000e+03]
Matrix condition number is high (49331252765552.74), adding regularization with epsilon 0.01.
matrix_3 Eigenvalues befo

In [21]:
# 优化均值计算
mean1 = np.mean(cov_matrix_1, axis=0)
mean2 = np.mean(cov_matrix_2, axis=0)
mean3 = np.mean(cov_matrix_3, axis=0)

# 动态调节 KL 散度中的 Trace 和均值差项
def dynamic_clip(value, threshold=250):
    """动态限制数值的最大值"""
    return min(value, threshold)

def kl_divergence(cov1, mean1, cov2, mean2, matrix_name=""):
    """
    计算两个多元高斯分布之间的KL散度
    """
    # 正定化矩阵
    cov2_regularized = regularize_matrix(cov2, matrix_name)
    
    # 检查特征值
    eigvals_cov2 = np.linalg.eigvals(cov2_regularized)
    if np.any(eigvals_cov2 <= 0):
        print(f"Matrix {matrix_name} has non-positive eigenvalues, skipping KL computation.")
        return np.nan
    
    # 使用标准逆矩阵
    try:
        inv_cov2 = np.linalg.inv(cov2_regularized)
    except np.linalg.LinAlgError:
        print(f"Matrix {matrix_name} inversion failed, skipping KL computation.")
        return np.nan
    
    # 计算 Trace 项
    term1 = np.trace(inv_cov2 @ cov1)
    term1 = dynamic_clip(term1, 250)  # 限制 Trace 项
    print(f"{matrix_name} KL term1 (Trace): {term1}")
    
    # 计算行列式对数差
    sign1, logdet1 = np.linalg.slogdet(cov1)
    sign2, logdet2 = np.linalg.slogdet(cov2_regularized)
    
    if sign1 <= 0 or sign2 <= 0:
        print(f"Matrix {matrix_name} determinant is non-positive.")
        return np.nan
    
    term2 = logdet2 - logdet1
    print(f"{matrix_name} KL term2 (Log determinant difference): {term2}")
    
    # 计算均值差项
    term3 = (mean2 - mean1).T @ inv_cov2 @ (mean2 - mean1)
    term3 = dynamic_clip(term3, 250)  # 限制均值差项
    print(f"{matrix_name} KL term3 (Mean difference): {term3}")
    
    # 计算 KL 散度
    result = 0.5 * (term1 + term2 + term3 - cov1.shape[0])
    print(f"{matrix_name} KL result: {result}")
    
    return max(result, 0)

# 计算并输出每两个矩阵之间的KL散度
kl_12 = kl_divergence(cov_matrix_1, mean1, cov_matrix_2, mean2, "matrix_12")
kl_13 = kl_divergence(cov_matrix_1, mean1, cov_matrix_3, mean3, "matrix_13")
kl_23 = kl_divergence(cov_matrix_2, mean2, cov_matrix_3, mean3, "matrix_23")

print(f"KL Divergence between matrix 1 and matrix 2: {kl_12}")
print(f"KL Divergence between matrix 1 and matrix 3: {kl_13}")
print(f"KL Divergence between matrix 2 and matrix 3: {kl_23}")

matrix_12 Eigenvalues before clipping: [2.00000000e-03 2.00000000e-03 2.00000000e-03 2.00000000e-03
 2.00000000e-03 2.00000000e-03 8.68579810e+01 9.75240978e+01
 1.00001000e+02]
matrix_12 Eigenvalues after clipping: [1.00000000e-02 1.00000000e-02 1.00000000e-02 1.00000000e-02
 1.00000000e-02 1.00000000e-02 8.68579810e+01 9.75240978e+01
 1.00001000e+02]
matrix_12 KL term1 (Trace): 0.609641976078252
matrix_12 KL term2 (Log determinant difference): 40.043298703166606
matrix_12 KL term3 (Mean difference): 1.8757419818071956
matrix_12 KL result: 16.764341330526026
matrix_13 Eigenvalues before clipping: [0.001001   0.001001   0.001001   0.001001   0.001001   0.001001
 0.0014363  0.00272776 0.88054957]
matrix_13 Eigenvalues after clipping: [0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.88054957]
matrix_13 KL term1 (Trace): 2.01620471533591
matrix_13 KL term2 (Log determinant difference): 17.056195207568464
matrix_13 KL term3 (Mean difference): 0.0
matrix